In [11]:
!conda install -y rasterio


Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::python-fastjsonschema==2.15.3=pyhd8ed1ab_0
  - conda-forge/noarch::bleach==5.0.0=pyhd8ed1ab_0
  - conda-forge/noarch::importlib_resources==5.7.1=pyhd8ed1ab_1
  - conda-forge/noarch::defusedxml==0.7.1=pyhd8ed1ab_0
  - conda-forge/linux-64::ipython==8.4.0=py310hff52083_0
  - conda-forge/noarch::prompt-toolkit==3.0.29=pyha770c72_0
  - conda-forge/linux-64::jedi==0.18.1=py310hff52083_1
  - conda-forge/noarch::packaging==21.3=pyhd8ed1ab_0
  - conda-forge/noarch::websocket-client==1.3.2=pyhd8ed1ab_0
  - conda-forge/linux-64::readline==8.1=h46c0cb4_0
  - conda-forge/noarch::parso==0.8.3=pyhd8ed1ab_0
  - conda-forge/noarch::nbconvert-core==6.5.0=pyhd8ed1ab_0
  - conda-forge/noarch::nbclient==0.6.4=pyhd8ed1ab_1
  - conda-forge/noarch::alembic==1.8.0=pyhd8ed1ab_0
  - conda-forge/noarch::pycparser==2.21=pyhd8ed1ab_0
  - con

In [14]:
!pip install loguru pyresample pyqt5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 11.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 13.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.4/266.4 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 14.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.7/359.7 kB 9.1 MB/s eta 0:00:0000:01
  Created wheel for pyresample: filename=pyresample-1.23.0-cp310-cp310-linux_x86_64.whl size=2352751 sha256=139e8e38b353fed7ea7ff7f144fd865620574cea681f0203d6b78d1f5cff385d
  Stored in directory: /home/jovyan/.cache/pip/wheels/22/d8/b7/bcf53169ccfa7e24d4cc9543d445f930520a75986444641a79
  Created wheel for c

In [15]:
import rasterio
import pyproj
from loguru import logger
from numpy import NaN, float32, save, meshgrid, zeros_like
#from icedrift.icedrift_utils import nan_to_zero

import sys
import datetime
import pathlib
import warnings


In [26]:
%store -r u
%store -r v
%store -r y1pm
%store -r x1pm
%store -r crs

In [35]:
print(y1pm)

[[ -22608.276589  -22608.276589  -22608.276589 ...  -22608.276589
   -22608.276589  -22608.276589]
 [ -27608.276589  -27608.276589  -27608.276589 ...  -27608.276589
   -27608.276589  -27608.276589]
 [ -32608.276589  -32608.276589  -32608.276589 ...  -32608.276589
   -32608.276589  -32608.276589]
 ...
 [-577608.276589 -577608.276589 -577608.276589 ... -577608.276589
  -577608.276589 -577608.276589]
 [-582608.276589 -582608.276589 -582608.276589 ... -582608.276589
  -582608.276589 -582608.276589]
 [-587608.276589 -587608.276589 -587608.276589 ... -587608.276589
  -587608.276589 -587608.276589]]


In [ ]:
dd

NameError: name 'A' is not defined

In [33]:
def write_geotiff_singleband(bands, band_names, coordinates, crs, params):
    """
    Writing Icedrift result arrays to separate GeoTiff files.

    Parameters
    ----------
    bands : list
        Containing a numpy.ndarray for each result band.
    band_names : list
        Containing str with names corresponding to 'bands'.
    #band_descriptions : list
        Containing str with descriptions corresponding to 'bands'.
    coordinates : list
        Containing two 1d lists as coordinate pairs: x_coordinates, y_coordinates.
        The length of the lists is equal to the dimensions (width, height) of the
        bands.
    crs : rasterio.crs.CRS
        Coordinate reference system information. Object includes exporting options
        as: crs.to_proj() or crs.to_wkt().
    #params : dict
        Icedrift processing parameters.
    """
    for i in range(len(bands)):
        data = bands[i]
        data_name = band_names[i]
        #data_description = band_descriptions[i]

        if data.dtype != 'float32':
            data = float32(data)

        geotiff_name = (f"-{data_name}.tif")

        x_coordinates, y_coordinates = coordinates
        transform = rasterio.transform.from_origin(
            x_coordinates[0] - 0.5 * (x_coordinates[1] - x_coordinates[0]),
            y_coordinates[0] - 0.5 * (y_coordinates[1] - y_coordinates[0]),
            x_coordinates[1] - x_coordinates[0],
            y_coordinates[0] - y_coordinates[1],
        )

        with rasterio.open(
            geotiff_name,
            "w",
            driver="GTiff",
            height=data.shape[0],
            width=data.shape[1],
            count=1,
            dtype=data.dtype,
            crs=crs,
            transform=transform,
            nodata=0.0,
        ) as dst:
            # Adding metadata
            #dst.set_band_description(1, data_description)
            dst.update_tags(data=data_name)
            dst.update_tags(dataset1=params["dataset1"])
            dst.update_tags(dataset2=params["dataset2"])
            dst.update_tags(time_gap=params["time_gap"])
            dst.update_tags(run_name=params["run_name"])
            dst.update_tags(cascades=len(params["cascade_levels"]))
            dst.update_tags(pyramids=len(params["pyramid_levels"]))
            dst.update_tags(min_template_shape=params["min_template_shape"])
            dst.update_tags(overlap_factor=params["overlap_factor"])
            dst.update_tags(step_size=params["stepsize"])
            dst.update_tags(cascade_levels=params["cascade_levels"])
            dst.update_tags(pyramid_levels=params["pyramid_levels"])
            dst.update_tags(min_window_shape=params["min_window_shape"])
            dst.update_tags(cores=params["cores"])
            dst.update_tags(correlation_threshold=params["correlation_threshold"])

            # Actual writing
            dst.write(data, 1)

        logger.info(
            "Data {} saved in singleband GeoTIFF format to {}".format(
                data_name, geotiff_name
            )
        )


In [34]:
write_geotiff_singleband((u,v), ('u','v'), (x1pm,y1pm), crs, 'NERSC')

IndexError: tuple index out of range

In [ ]:
def write(bands, band_names, band_descriptions, coordinates, crs, parameters, output_format):
    """
    Switcher for different writing options. 

    Parameters
    ----------
    bands : list
        Containing a numpy.ndarray for each result band.
    band_names : list
        Containing str with names corresponding to 'bands'.
    band_descriptions : list
        Containing str with descriptions corresponding to 'bands'.
    coordinates : list
        Containing two 1d lists as coordinate pairs: x_coordinates, y_coordinates.
        The length of the lists is equal to the dimensions (width, height) of the
        bands.
    crs : rasterio.crs.CRS
        Coordinate reference system information. Object includes exporting options
        as: crs.to_proj() or crs.to_wkt().
    params : dict
        Icedrift processing parameters.
    output_format : str
        See 'Returns' for options.

    Returns
    -------
    function
        Dependent on the 'output_format' a different function is called:
            'singleband_geotiff'    write_geotiff_singleband
            'multiband_geotiff'     write_geotiff_multiband
            'singleband_netcdf'     write_netCDF_singleband
            'multiband_netcdf'      write_netCDF_multiband
            'singleband_binary'     write_ENVI_singleband        
    """
    if output_format == None:
        logger.info('No output will be written')
        return
    else:
        switcher = {
            'singleband_geotiff': write_geotiff_singleband,
            'multiband_geotiff': write_geotiff_multiband,
            'singleband_netcdf': write_netCDF_singleband,
            'multiband_netcdf': write_netCDF_multiband,
            'singleband_binary': write_ENVI_singleband,
        }
        # Get the function from switcher dictionary
        writing_method = switcher.get(output_format)
        # Execute the function
        return writing_method(bands, band_names, band_descriptions, coordinates, crs, parameters)
